## OpenAI various API's
Make sure you sign up for OpenAI's dev account and setup your API Access
Tutorial on setting this up coming soon

In [1]:
#!pip install openai-whisper
#!pip install ffmpeg
#!pip install openai

In [2]:
OPENAI_API_KEY = 'YOUROPENAIAPIKEY'
OPENAI_ORG_ID = 'YOUROPENAIORGID'

# OpenAI Text to Speech
Quickstart Docs [here](https://platform.openai.com/docs/guides/text-to-speech/quickstart)

In [58]:
from pathlib import Path
from openai import OpenAI
from IPython.display import Audio

In [59]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

In [71]:
filename = 'output.mp3'
speech_file_path = Path(filename)

In [72]:
speech_file_path

PosixPath('output.mp3')

In [ ]:
response = client.audio.speech.create(
    model="tts-1",
    voice="Nova",
    input="Hello world! This is an OpenAI TTS test.",
)
response.stream_to_file(speech_file_path) # response needs to be updated to avoid the warning below

In [66]:
display(Audio(filename, autoplay=True))

## Using HD voices

In [73]:
filename = 'output2.mp3'
speech_file_path = Path(filename)

response = client.audio.speech.create(
    model="tts-1-hd",
    voice="alloy",
    input="Hello world! This is an OpenAI TTS test.",
)
response.stream_to_file(speech_file_path) # response needs to be updated to avoid the warning below

/tmp/ipykernel_18364/130479104.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path) # response needs to be updated to avoid the warning below


In [74]:
display(Audio(filename, autoplay=True))

## Other languages using HD TTS

In [75]:
myText = '이 노래는 누가 더 잘못했고 그런 건 어젯밤까지 나 주묘했었지는 더 쓸 때'

In [80]:
filename = 'output3.mp3'
speech_file_path = Path(filename)

response = client.audio.speech.create(
    model="tts-1-hd",
    voice="nova",
    input=myText,
)
response.stream_to_file(speech_file_path) # response needs to be updated to avoid the warning below

/tmp/ipykernel_18364/4009023941.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path) # response needs to be updated to avoid the warning below


In [81]:
display(Audio(filename, autoplay=True))

# OpenAI Speech to text and using GPT to translate
Note: You will need to download your favorite KRap/Kpop song. </br>
I used music from the amazing [Heize](https://www.youtube.com/@Heizeofficial) </br>
This is the song: [헤이즈(Heize) - 그러니까(So, it ends?)(feat.Colde)](https://www.youtube.com/watch?v=TMWXNueRYlg)

### Whisper API

In [3]:
import whisper

In [4]:
model = whisper.load_model("base")

In [7]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("Heize So it ends feat Colde.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

Detected language: ko
이 노래는 누가 더 잘못했고 그런 건 어젯밤까지 나 주묘했었지는 더 쓸 때


### Note:
If you get an ffmpeg error or `FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'` make sure you install ffmpeg in your os or env

In [8]:
result.text

'이 노래는 누가 더 잘못했고 그런 건 어젯밤까지 나 주묘했었지는 더 쓸 때'

# GPT 3.5 API 

### New OpenAI API

In [19]:
import os
from openai import OpenAI

In [25]:
openai.organization = OPENAI_ORG_ID
openai.api_key = OPENAI_API_KEY

In [27]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

In [32]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)
print(chat_completion.choices[0].message.content)

This is a test.


In [36]:
prompt = "Translate this into English: " + result.text
prompt

'Translate this into English: 이 노래는 누가 더 잘못했고 그런 건 어젯밤까지 나 주묘했었지는 더 쓸 때'

In [42]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": '{prompt}',
        }
    ],
    model="gpt-3.5-turbo",
)

In [43]:
print(result.text, '\ntranslates to:\n', chat_completion.choices[0].message.content)

이 노래는 누가 더 잘못했고 그런 건 어젯밤까지 나 주묘했었지는 더 쓸 때 
translates to:
 Please provide a writing prompt for me to help you with.


### For older versions of OpenAI API <= 0.28

In [ ]:
import os
import openai
openai.organization = OPENAI_ORG_ID
openai.api_key = OPENAI_API_KEY
#openai.Model.list()

In [ ]:
#For older version of OpenAI:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Translate this into English: " + result.text,
  temperature=0.3,
  max_tokens=100,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

In [26]:
response.choices

[<OpenAIObject at 0x7f31aea939f0> JSON: {
   "finish_reason": "stop",
   "index": 0,
   "logprobs": null,
   "text": "\uac00 \uc788\uc5c8\uc5b4\n\nThis song was about who was wrong and until last night I was wishing there was more time."
 }]